In [5]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import StructuredTool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(a, b):
#     return a + b


# # structured input ReAct(Reasoning and Acting) -> not related to reactjs lol
# # * how an agent works like a loop and langchain does pass the context.
# # langchane behaves like a bridge -> sometimes it can be broken
# # -> better option: OpenAI function calling -> force the output
# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     tools=[
#         StructuredTool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Both arguments must be numbers.",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [6]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import Tool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(inputs):
#     a, b = inputs.split(",")
#     return float(a) + float(b)


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
#     # but it is recommended for new use cases to be built with LangGraph.
#     # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     # It works well now without setting True
#     handle_parsing_errors=True,
#     tools=[
#         Tool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Use this tool by sending a pair of numbers separated by a comma. \n Example:1,2",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [7]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


# define args types by using pydantic
class CalculatorToolArgsSchema(BaseModel):
    # if set int, langchain automatically rounds values
    a: float = Field(discription="The first number")
    b: float = Field(discription="The second number")


class CalculatorTool(BaseTool):

    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    the first and second arguments must be numbers
    only receive two numbers.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    # now the data structure is simple, but if prepare for bit data structures,
    # separating function is better than just repeat a + b in return
    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
    # but it is recommended for new use cases to be built with LangGraph.
    # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
    agent=AgentType.OPENAI_FUNCTIONS,
    # It works well now without setting True
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}